In [107]:
import matplotlib.pyplot as plt
import pandas as pd

In [108]:
df = pd.read_csv("../data/completo/dataset.csv")
df.head(10)

,Unnamed: 0,jurisdicción,región,año,mes,focosCalor,hectareas,incendiosReportados
0,0,Buenos Aires,Pampeana,2017,1,2844,52324.0,37
1,1,Buenos Aires,Pampeana,2017,2,672,52324.0,37
2,2,Buenos Aires,Pampeana,2017,3,597,52324.0,37
3,3,Buenos Aires,Pampeana,2017,4,494,52324.0,37
4,4,Buenos Aires,Pampeana,2017,5,353,52324.0,37
5,5,Buenos Aires,Pampeana,2017,6,415,52324.0,37
6,6,Buenos Aires,Pampeana,2017,7,449,52324.0,37
7,7,Buenos Aires,Pampeana,2017,8,564,52324.0,37
8,8,Buenos Aires,Pampeana,2017,9,603,52324.0,37
9,9,Buenos Aires,Pampeana,2017,10,481,52324.0,37


In [32]:
df.shape

(2304, 8)

In [78]:
import plotly.express as px


# ! Focos por Año

focos_anual = df.groupby('año')['focosCalor'].sum().reset_index()


fig = px.line(focos_anual, x='año', 
              y='focosCalor',
              labels={'año': 'Año', 'focosCalor': 'Focos de Calor'},
              
              markers= True)
fig.update_traces(
    marker=dict(
        size=10, 
    ),
    line=dict(width=2.5, color='blue'), 
    textposition='top center'  
)
fig.write_html("../web/public/focos_año.html", full_html=False)
fig.show()


In [86]:

# ! Focos por año y jurisdicción


focos_prov_año = df.groupby(['jurisdicción', 'año'])['focosCalor'].sum().unstack(fill_value=0)


fig = px.imshow(focos_prov_año,
                labels={'x': 'Año', 'y': 'Jurisdicción', 'color': 'Focos'},
                aspect='auto',
                color_continuous_scale='YlOrRd')

fig.write_html("../web/public/focos_prov_año.html", full_html=False)
fig.show()


In [109]:

# ! Focos por mes y jurisdicción

focos_prov_mes = df.groupby(['jurisdicción', 'mes'])['focosCalor'].sum().unstack(fill_value=0)

fig = px.imshow(focos_prov_mes,
                labels={'x': 'Mes', 'y': 'Jurisdicción', 'color': 'Focos'},
                aspect='auto',
                color_continuous_scale='YlOrRd')

fig.update_xaxes(
    tickmode='linear',
    tick0=1,
    dtick=1
)

fig.write_html("../web/public/focos_prov_mes.html", full_html=False)
fig.show()



In [ ]:
import pandas as pd
import plotly.graph_objects as go

# ! Superficie afectada

df = pd.read_csv('../data/limpio/c_hect_año.csv')


fig = go.Figure(data=go.Heatmap(
    z=df.set_index('Jurisdicción').values,
    x=df.columns[1:],
    y=df['Jurisdicción'],
    colorscale='YlOrRd',
    hoverongaps=False,
   
    texttemplate="%{text:,.0f}",
    hovertemplate="<b>Jurisdicción:</b> %{y}<br>" +
                 "<b>Año:</b> %{x}<br>" +
                 "<b>Superficie afectada:</b> %{z:,.0f} ha<extra></extra>",
    colorbar=dict(title='Hectáreas')
))


fig.update_layout(
    
    xaxis_title='Año',
    yaxis_title='Jurisdicción',
    height=700,
    width=900,
    yaxis=dict(autorange='reversed'),
    hoverlabel=dict(
        bgcolor="white",
        font_size=14,
        font_family="Arial"
    )
)

fig.write_html("../web/public/superficie.html", full_html=False)
fig.show()

In [70]:
# ! Incendios por Año y Mes (Barras Agrupadas)

df = pd.read_csv('../data/limpio/c_incendios_mes.csv')

df = df.drop('2025', axis=1)
meses_orden = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 
               'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
df['Mes'] = pd.Categorical(df['Mes'], categories=meses_orden, ordered=True)
df = df.sort_values('Mes')

df_melted = df.melt(id_vars='Mes', var_name='Año', value_name='Incendios')

fig = px.bar(
    df_melted,
    x='Mes', 
    y='Incendios',
    color='Año',  
    labels={'Incendios': 'Número de incendios', 'Mes': 'Mes'},
    category_orders={'Mes': meses_orden},
    color_discrete_sequence=px.colors.qualitative.Plotly
)

fig.update_layout(
    barmode='group', 
    height=600,
    xaxis={'type': 'category', 'title': 'Mes'},
    yaxis={'title': 'Número de incendios', 'rangemode': 'tozero'},
    legend_title='Año',
    hovermode='x unified',
)

fig.update_traces(
    hovertemplate="<b>Año %{data.name}</b><br>Mes: %{x}<br>Incendios: %{y:,}<extra></extra>"
)

fig.write_html("../web/public/incendios_mes.html", full_html=False)
fig.show()

In [73]:

# ! Incendios por Región

import plotly.graph_objects as go


df_region = pd.read_csv('../data/limpio/c_incendios_region.csv')

df_region = df_region.drop('2025', axis=1)

years = df_region.columns[1:]
regiones = df_region['Región']


fig = go.Figure()


for i, year in enumerate(years):
    fig.add_trace(go.Bar(
        x=df_region['Región'],
        y=df_region[year],
        name=year,
        hovertemplate=f"Año: {year}<br>Región: %{{x}}<br>Incendios: %{{y:,}}<extra></extra>"
    ))

fig.update_layout(
    xaxis_title='Región',
    yaxis_title='Número de Incendios',
    barmode='group',  # ! Este es el parámetro para agrupar las barras
    template='plotly_white',
    legend_title='Año',
    height=600,
    xaxis={'categoryorder': 'total descending'},
    hovermode='x unified'
)


if len(years) > 5:
    fig.update_layout(bargap=0.2, bargroupgap=0.1)
fig.write_html("../web/public/incendios_region.html", full_html=False)
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# ! Incendios por Provincia y Año
df = pd.read_csv('../data/limpio/in_prov_año.csv')


fig = px.imshow(
    df.set_index('Jurisdicción'),
    labels=dict(x="Año", y="Provincia", color="Incendios"),
    color_continuous_scale='YlOrRd',
    aspect="auto",
)


fig.update_layout(
    height=800,
    coloraxis_colorbar={'title': 'Incendios'}
    
)
fig.write_html("../web/public/incendios_prov_año.html", full_html=False)
fig.show()